# Pandas for Data Analysis



This Notebook collects some best practices of using Python Pandas library for data analysis. 

Something to get from this notes:

- `%%time` function
- indexing
    + `set_index`
    + `sort_index`
- method chaining and the power of `agg()`
- following frequently used methods:
    + `query` - the pd version of `filter` in dplyr
    + `groupby` - the pd version of `group_by` in dplyr
    + `agg` - the pd version of `summarise` in dplyr
- use methods within `agg`
- pipes + self-defined log functions

In [1]:
import pandas as pd
import numpy as np
import os

Current `pandas` version 0.25.0

According to [this](https://medium.com/unit8-machine-learning-publication/from-pandas-wan-to-pandas-master-4860cf0ce442) post: "The next release v0.25 is set to be released in July 2019 ( v0.25rc0 has been released on the 4th July) and it is the same codebase as v1.0 but with warning messages for soon-to-be-deprecated methods. So, if you are planning to go for v1.0, you should be careful with all the deprecation warnings when you run your codebase with v0.25."

In [2]:
assert pd.__version__ == '0.25.0'

Data is downloaded from [kaggle](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016/downloads/suicide-rates-overview-1985-to-2016.zip/1). 

In [3]:
%%time

data_path = '../data/'

df = (pd.read_csv(filepath_or_buffer=os.path.join(data_path, 'suicide_rates.csv'))
      .rename(columns={'suicides/100k pop' : 'suicides_per_100k',
                       ' gdp_for_year ($) ' : 'gdp_year', 
                       'gdp_per_capita ($)' : 'gdp_capita',
                       'country-year' : 'country_year'})
      .assign(gdp_year=lambda _df: _df['gdp_year'].str.replace(',','').astype(np.int64))
     )

CPU times: user 70.8 ms, sys: 16.7 ms, total: 87.6 ms
Wall time: 98.2 ms


In [4]:
df.head()

,country,year,sex,age,suicides_no,population,suicides_per_100k,country_year,HDI for year,gdp_year,gdp_capita,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,2156624900,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers


`describe()` function to output different statistics for every **numeric** column

In [5]:
df.describe()

,year,suicides_no,population,suicides_per_100k,HDI for year,gdp_year,gdp_capita
count,27820.000000,27820.000000,2.782000e+04,27820.000000,8364.000000,2.782000e+04,27820.000000
mean,2001.258375,242.574407,1.844794e+06,12.816097,0.776601,4.455810e+11,16866.464414
std,8.469055,902.047917,3.911779e+06,18.961511,0.093367,1.453610e+12,18887.576472
min,1985.000000,0.000000,2.780000e+02,0.000000,0.483000,4.691962e+07,251.000000
25%,1995.000000,3.000000,9.749850e+04,0.920000,0.713000,8.985353e+09,3447.000000
50%,2002.000000,25.000000,4.301500e+05,5.990000,0.779000,4.811469e+10,9372.000000
75%,2008.000000,131.000000,1.486143e+06,16.620000,0.855000,2.602024e+11,24874.000000
max,2016.000000,22338.000000,4.380521e+07,224.970000,0.944000,1.812071e+13,126352.000000


In [6]:
df.describe(include = 'all')

,country,year,sex,age,suicides_no,population,suicides_per_100k,country_year,HDI for year,gdp_year,gdp_capita,generation
count,27820,27820.000000,27820,27820,27820.000000,2.782000e+04,27820.000000,27820,8364.000000,2.782000e+04,27820.000000,27820
unique,101,NaN,2,6,NaN,NaN,NaN,2321,NaN,NaN,NaN,6
top,Mauritius,NaN,male,75+ years,NaN,NaN,NaN,Kazakhstan1997,NaN,NaN,NaN,Generation X
freq,382,NaN,13910,4642,NaN,NaN,NaN,12,NaN,NaN,NaN,6408
mean,NaN,2001.258375,NaN,NaN,242.574407,1.844794e+06,12.816097,NaN,0.776601,4.455810e+11,16866.464414,NaN
std,NaN,8.469055,NaN,NaN,902.047917,3.911779e+06,18.961511,NaN,0.093367,1.453610e+12,18887.576472,NaN
min,NaN,1985.000000,NaN,NaN,0.000000,2.780000e+02,0.000000,NaN,0.483000,4.691962e+07,251.000000,NaN
25%,NaN,1995.000000,NaN,NaN,3.000000,9.749850e+04,0.920000,NaN,0.713000,8.985353e+09,3447.000000,NaN
50%,NaN,2002.000000,NaN,NaN,25.000000,4.301500e+05,5.990000,NaN,0.779000,4.811469e+10,9372.000000,NaN
75%,NaN,2008.000000,NaN,NaN,131.000000,1.486143e+06,16.620000,NaN,0.855000,2.602024e+11,24874.000000,NaN


Output all the unique values of a variable:

In [7]:
print(df['country'].nunique())

print(df['country'].unique())

101
['Albania' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia'
 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Barbados' 'Belarus' 'Belgium'
 'Belize' 'Bosnia and Herzegovina' 'Brazil' 'Bulgaria' 'Cabo Verde'
 'Canada' 'Chile' 'Colombia' 'Costa Rica' 'Croatia' 'Cuba' 'Cyprus'
 'Czech Republic' 'Denmark' 'Dominica' 'Ecuador' 'El Salvador' 'Estonia'
 'Fiji' 'Finland' 'France' 'Georgia' 'Germany' 'Greece' 'Grenada'
 'Guatemala' 'Guyana' 'Hungary' 'Iceland' 'Ireland' 'Israel' 'Italy'
 'Jamaica' 'Japan' 'Kazakhstan' 'Kiribati' 'Kuwait' 'Kyrgyzstan' 'Latvia'
 'Lithuania' 'Luxembourg' 'Macau' 'Maldives' 'Malta' 'Mauritius' 'Mexico'
 'Mongolia' 'Montenegro' 'Netherlands' 'New Zealand' 'Nicaragua' 'Norway'
 'Oman' 'Panama' 'Paraguay' 'Philippines' 'Poland' 'Portugal' 'Puerto Rico'
 'Qatar' 'Republic of Korea' 'Romania' 'Russian Federation'
 'Saint Kitts and Nevis' 'Saint Lucia' 'Saint Vincent and Grenadines'
 'San Marino' 'Serbia' 'Seychelles' 'Singapore' 'Slovakia' 'Slovenia'
 'So

List all available methods for DataFrame `df`:

In [8]:
dir(df)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 

### I. Indexing

In [9]:
%%time
mi_df = df.set_index(['country', 'year', 'sex', 'age'], drop=False)  # drop = False allows to not drop the columns used as the new index

CPU times: user 13.8 ms, sys: 3.26 ms, total: 17 ms
Wall time: 25.9 ms


In [10]:
%%time
print(mi_df.loc['Albania', 1987, 'male', '25-34 years'])

country                  Albania
year                        1987
sex                         male
age                  25-34 years
suicides_no                    9
population                274300
suicides_per_100k           3.28
country_year         Albania1987
HDI for year                 NaN
gdp_year              2156624900
gdp_capita                   796
generation               Boomers
Name: (Albania, 1987, male, 25-34 years), dtype: object
CPU times: user 4.91 ms, sys: 2.85 ms, total: 7.76 ms
Wall time: 6.03 ms


In [11]:
%%time
mi_df.sort_index()

CPU times: user 59.4 ms, sys: 4.84 ms, total: 64.2 ms
Wall time: 85.2 ms


country  year     sex          age  \
country    year sex    age                                                  
Albania    1987 female 15-24 years     Albania  1987  female  15-24 years   
                       25-34 years     Albania  1987  female  25-34 years   
                       35-54 years     Albania  1987  female  35-54 years   
                       5-14 years      Albania  1987  female   5-14 years   
                       55-74 years     Albania  1987  female  55-74 years   
...                                        ...   ...     ...          ...   
Uzbekistan 2014 male   25-34 years  Uzbekistan  2014    male  25-34 years   
                       35-54 years  Uzbekistan  2014    male  35-54 years   
                       5-14 years   Uzbekistan  2014    male   5-14 years   
                       55-74 years  Uzbekistan  2014    male  55-74 years   
                       75+ years    Uzbekistan  2014    male    75+ years   

                                    suicides_no  population  \
country    year sex    age                                    
Albania    1987 female 15-24 years           14      289700   
                       25-34 years            4      257200   
                       35-54 years            6      278800   
                       5-14 years             0      311000   
                       55-74 years            0      144600   
...                                         ...         ...   
Uzbekistan 2014 male   25-34 years          318     2739150   
                       35-54 years          519     3421300   
                       5-14 years            60     2762158   
                       55-74 years          144     1271111   
                       75+ years             17      224995   

                                    suicides_per_100k    country_year  \
country    year sex    age                                              
Albania    1987 female 15-24 years               4.83     Albania1987   
                       25-34 years               1.56     Albania1987   
                       35-54 years               2.15     Albania1987   
                       5-14 years                0.00     Albania1987   
                       55-74 years               0.00     Albania1987   
...                                               ...             ...   
Uzbekistan 2014 male   25-34 years              11.61  Uzbekistan2014   
                       35-54 years              15.17  Uzbekistan2014   
                       5-14 years                2.17  Uzbekistan2014   
                       55-74 years              11.33  Uzbekistan2014   
                       75+ years                 7.56  Uzbekistan2014   

                                    HDI for year     gdp_year  gdp_capita  \
country    year sex    age                                                  
Albania    1987 female 15-24 years           NaN   2156624900         796   
                       25-34 years           NaN   2156624900         796   
                       35-54 years           NaN   2156624900         796   
                       5-14 years            NaN   2156624900         796   
                       55-74 years           NaN   2156624900         796   
...                                          ...          ...         ...   
Uzbekistan 2014 male   25-34 years         0.675  63067077179        2309   
                       35-54 years         0.675  63067077179        2309   
                       5-14 years          0.675  63067077179        2309   
                       55-74 years         0.675  63067077179        2309   
                       75+ years           0.675  63067077179        2309   

                                         generation  
country    year sex    age                           
Albania    1987 female 15-24 years     Generation X  
                       25-34 years          Boomers  
                       35-54 years           Sile

In [12]:
%%time
mi_df.index.is_monotonic

CPU times: user 46.6 ms, sys: 2.95 ms, total: 49.5 ms
Wall time: 59.5 ms


False

### II. Method chaining

#### 1. Show unique values of generation by age group:

In [13]:
# add an overall parenthesis around the chain to allow to start new line
df_unique_generation = (df
                        .groupby(['age'])
                        .agg(unique_generation = ('generation', 'unique')))  # recommended from v0.25

## another way to do for older versions:
# (df
# .groupby(['age'])
# .agg({'generation' : 'unique'})
# .rename(columns = {'generation':'unique_generation'}))

df_unique_generation

,unique_generation
age,
15-24 years,"[Generation X, Millenials]"
25-34 years,"[Boomers, Generation X, Millenials]"
35-54 years,"[Silent, Boomers, Generation X]"
5-14 years,"[Generation X, Millenials, Generation Z]"
55-74 years,"[G.I. Generation, Silent, Boomers]"
75+ years,"[G.I. Generation, Silent]"


In [14]:
df_unique_generation['unique_generation']

age
15-24 years                  [Generation X, Millenials]
25-34 years         [Boomers, Generation X, Millenials]
35-54 years             [Silent, Boomers, Generation X]
5-14 years     [Generation X, Millenials, Generation Z]
55-74 years          [G.I. Generation, Silent, Boomers]
75+ years                     [G.I. Generation, Silent]
Name: unique_generation, dtype: object

#### 2. Show total number of suicides by `year` and `country`

In [15]:
df_suicides_summary = (df
 .groupby(['country', 'year'])
 .agg(total_suicides_per_100k = ('suicides_per_100k', 'sum'))
 .nlargest(10, columns = 'total_suicides_per_100k')
)

Notice that the groupby variables has been turned into indices, hence not appear in the output DataFrame. This is different from group_by + summarise in R dplyr.

In [16]:
df_suicides_summary.shape

(10, 1)

To get a similar output as R, we can use the following method to `reset_index()`:

In [17]:
df_suicides_summary1 = (df
 .groupby(['country', 'year'])
 .agg(total_suicides_per_100k = ('suicides_per_100k', 'sum'))
 .nlargest(10, columns = 'total_suicides_per_100k')
 .reset_index()
)

df_suicides_summary1

,country,year,total_suicides_per_100k
0,Lithuania,1995,639.30
1,Lithuania,1996,595.61
2,Hungary,1991,575.00
3,Lithuania,2000,571.80
4,Hungary,1992,570.26
5,Lithuania,2001,568.98
6,Russian Federation,1994,567.64
7,Lithuania,1998,566.36
8,Lithuania,1997,565.44
9,Lithuania,1999,561.53


This is another popular solution from [Stackoverflow](https://stackoverflow.com/questions/32059397/pandas-groupby-without-turning-grouped-by-column-into-index), but not quite working for my case:

In [18]:
df_suicides_summary2 = (df
 .groupby(['country', 'year'], as_index = False)
 .agg(total_suicides_per_100k = ('suicides_per_100k', 'sum'))
 .nlargest(10, columns = 'total_suicides_per_100k')
)

In [19]:
df_suicides_summary2

,total_suicides_per_100k
1255,639.30
1256,595.61
948,575.00
1260,571.80
949,570.26
1261,568.98
1752,567.64
1258,566.36
1257,565.44
1259,561.53


### III. Use pipe 

#### III.a Use pipe to log different information

In [20]:
def log_head(df, head_count=10):
    print(df.head(head_count))
    return df

def log_columns(df):
    print(df.columns)
    return df

def log_shape(df):
    print(f'shape = {df.shape}')
    return df

In [21]:
(df
 .pipe(log_shape)
 .query('sex == "female"')
# for one time query, the query function is actually easier than set_index + loc 
#  .set_index(['sex'])
#  .loc["female"]
 .groupby(['year', 'country'])
 .agg(sum_suicides_per_100k_female = ('suicides_per_100k', 'sum'))
 .reset_index()
 .pipe(log_shape)
 .pipe(log_columns)
 .nlargest(n=10, columns=['sum_suicides_per_100k_female'])
)

shape = (27820, 12)
shape = (2321, 3)
Index(['year', 'country', 'sum_suicides_per_100k_female'], dtype='object')


,year,country,sum_suicides_per_100k_female
1805,2009,Republic of Korea,170.89
241,1989,Singapore,163.16
87,1986,Singapore,161.67
1893,2010,Republic of Korea,158.52
1634,2007,Republic of Korea,149.60
1982,2011,Republic of Korea,147.84
342,1991,Hungary,147.35
1718,2008,Republic of Korea,147.04
975,2000,Aruba,146.22
1460,2005,Republic of Korea,145.35


#### III.b Use pipe and apply functions directly on DataFrame's columns

In [22]:
from sklearn.preprocessing import MinMaxScaler

def norm_df(df, columns):
    return df.assign(**{col: MinMaxScaler().fit_transform(df[[col]].values.astype(float)) 
                        for col in columns})
                        
    
for sex in ['male', 'female']:
    print(sex)
    print(
        df
        .query(f'sex == "{sex}"')
        .groupby(['country'])
        .agg(suicides_per_100k_sum = ('suicides_per_100k', 'sum'),
             gdp_year_mean = ('gdp_year', 'mean'))
        .pipe(norm_df, columns=['suicides_per_100k_sum', 'gdp_year_mean'])
        .corr(method='spearman')
    )
    print('\n')

male
                       suicides_per_100k_sum  gdp_year_mean
suicides_per_100k_sum               1.000000       0.421218
gdp_year_mean                       0.421218       1.000000


female
                       suicides_per_100k_sum  gdp_year_mean
suicides_per_100k_sum               1.000000       0.452343
gdp_year_mean                       0.452343       1.000000




### References:

- Medium post: https://medium.com/unit8-machine-learning-publication/from-pandas-wan-to-pandas-master-4860cf0ce442